# Outils-corpus 5
## [Spacy](https://spacy.io)

- Bibliothèque logicielle de TAL écrite en Python (et Cython)
- Étiquetage POS, lemmatisation, analyse syntaxique, entités nommées, word embedding, transformers
- Usage de modèles neuronaux
- Intégration aisée de bibliothèques de deep learning
- v3.0.3 ([github](https://github.com/explosion/spaCy))
- Licence MIT (Open Source) pour le code
    - Licences ouvertes diverses pour les modèles
- Produit de la société [explosion.ai](https://explosion.ai/). Fondé par : Matthew Honnibal ([@honnibal](https://twitter.com/honnibal)) et Ines Montani ([@_inesmontani](https://twitter.com/_inesmontani))

## Pourquoi Spacy ?

- C'est du Python 🙌 🎉
- Plutôt simple à prendre en main
- Très bien documenté, à notre avis. D'ailleurs plutôt que ce notebook, suivez l'excellent tutorial d'Ines Montani : [https://course.spacy.io/](https://course.spacy.io/)
- Couvre les traitements d'une chaîne de TAL typique
- Pas mal utilisé dans l'industrie
- MAIS ce n'est pas forcément l'outil qui donne les meilleurs résultats pour le français dans toutes les tâches de TAL

## Spacy et les autres

Spacy est *un* des frameworks de TAL disponibles

- [NLTK](http://www.nltk.org/) : python, orienté pédagogie, pas de modèles neuronaux inclus mais se combine bien avec TensorFlow, PyTorch ou AlleNLP
- [Stanford Core NLP](https://stanfordnlp.github.io/stanfordnlp/) : java, modèles pour 53 langues (UD), résolution de la coréference.
- [Stanza](https://stanfordnlp.github.io/stanza/) : python, nouveau framework de Stanford, modèles neuronaux entraînés sur données UD <small>[https://github.com/explosion/spacy-stanza](https://github.com/explosion/spacy-stanza) permet d'utiliser les modèles de Stanford avec Spacy</small>
- [TextBlob](https://textblob.readthedocs.io/en/dev/)
- [DKPro](https://dkpro.github.io/)
- [flair](https://github.com/zalandoresearch/flair) : le framework de Zalando, très bonnes performances en reconnaissance d'entités nommées

## installation

dans un terminal
```bash
python3 -m pip install -U --user spacy 
#ou pip install -U --user spacy
```
- installation du modèle français
```bash
python3 -m spacy download fr_core_news_md
#ou python3 -m spacy download fr_core_news_sm 
```
- vérification
```bash
python3 -m spacy validate
```


## modèles

- Spacy utilise des modèles statistiques qui permettent de prédire des annotations linguistiques
- 16 langues : allemand, anglais, chinois, danois, espagnol, français, italien, japonais, lituanien, néerlandais, grec, norvégien, polonais, portugais, roumain, russe + modèle multi langues
- 4 modèles pour le français
    - fr_core_news_sm (tagger, morphologizer, lemmatizer, parser, ner) 16 Mo
    - fr_core_news_md (tagger, morphologizer, lemmatizer, parser, ner, vectors) 45 Mo
    - fr_core_news_lg (tagger, morphologizer, lemmatizer, parser, ner, vectors) 546 Mo
    - fr_dep_news_trf (tagger, morphologizer, lemmatizer, parser) 381 Mo
- modèles `fr` appris sur les corpus [Sequoia](https://deep-sequoia.inria.fr/fr/) et [WikiNer](https://figshare.com/articles/Learning_multilingual_named_entity_recognition_from_Wikipedia/5462500) sauf le modèle `trf` qui est issu de camembert-base distribué par [Hugging Face](https://huggingface.co/camembert-base).
- Tous ces modèles, quelque soient leur type ou leur langue, s'utilisent de la même façon, avec la même API.

## usage

- *si vous voulez utiliser Spacy prenez le temps de lire la [documentation](https://spacy.io/usage), ici ce ne sera qu'un coup d'œil incomplet*
- un modèle est une instance de la classe `Language`, il est adapté à une langue en particulier
- un modèle incorpore un vocabulaire, des poids, des vecteurs de mots, une configuration

In [21]:
import spacy
nlp = spacy.load('fr_core_news_md')

In [22]:
type(nlp)

spacy.lang.fr.French

- le traitement fonctionne avec un [*pipeline*](https://spacy.io/usage/spacy-101#pipelines) pour convertir un texte en objet `Doc` (texte annoté)
- par défaut `tokenizer` > `tagger` > `parser` > `ner` > `…`
- depuis la v3 le pipeline devient `tok2vec` > `morphologizer` > `parser` > `ner` > `attribute_ruler` > `lemmatizer`  
  ou `transformer` > `morphologizer` > `parser` > `ner` > `attribute_ruler` > `lemmatizer`
- l'utilisateur peut ajouter des étapes ou en retrancher

In [3]:
nlp = spacy.load('fr_core_news_md', disable=["parser", "ner"])
nlp.pipeline

[('tok2vec', <spacy.pipeline.tok2vec.Tok2Vec at 0x7f4854fb5a10>),
 ('morphologizer',
  <spacy.pipeline.morphologizer.Morphologizer at 0x7f4854f4ffb0>),
 ('attribute_ruler',
  <spacy.pipeline.attributeruler.AttributeRuler at 0x7f4854f1d7d0>),
 ('lemmatizer', <spacy.lang.fr.lemmatizer.FrenchLemmatizer at 0x7f4854f18fa0>)]

Retour au pipeline par défaut

In [4]:
nlp = spacy.load('fr_core_news_md')
nlp.pipeline

[('tok2vec', <spacy.pipeline.tok2vec.Tok2Vec at 0x7f4854f933b0>),
 ('morphologizer',
  <spacy.pipeline.morphologizer.Morphologizer at 0x7f4860c4e710>),
 ('parser', <spacy.pipeline.dep_parser.DependencyParser at 0x7f486273d600>),
 ('ner', <spacy.pipeline.ner.EntityRecognizer at 0x7f486273d520>),
 ('attribute_ruler',
  <spacy.pipeline.attributeruler.AttributeRuler at 0x7f4854f254b0>),
 ('lemmatizer', <spacy.lang.fr.lemmatizer.FrenchLemmatizer at 0x7f4847288a50>)]

 - Un objet `Doc` est une séquence d'objets `Token` (voir l'[API](https://spacy.io/api/token))
 - Le texte d'origine est découpé en phrases, tokenizé, annoté en POS, lemme, syntaxe (dépendance) et en entités nommées (NER)

In [23]:
doc = nlp("L’Organisation des Nations unies (ONU) a lancé mardi un appel d’urgence pour lever des dizaines de millions de dollars afin de protéger les réfugiés vulnérables face à la propagation du nouveau coronavirus.")
type(doc)

spacy.tokens.doc.Doc

## usage – tokenization

La tokenization de Spacy est non-destructive. Vous pouvez découper un texte en tokens et le restituer dans sa forme originale.

In [24]:
doc = nlp("L'Organisation des Nations unies (ONU) a lancé mardi un appel d'urgence pour lever des dizaines de millions de dollars afin de protéger les réfugiés vulnérables face à la propagation du nouveau coronavirus.")
for token in doc:
    print(token)

L'
Organisation
des
Nations
unies
(
ONU
)
a
lancé
mardi
un
appel
d'
urgence
pour
lever
des
dizaines
de
millions
de
dollars
afin
de
protéger
les
réfugiés
vulnérables
face
à
la
propagation
du
nouveau
coronavirus
.


In [7]:
for token in doc:
    print(token.text_with_ws, end="")

L'Organisation des Nations unies (ONU) a lancé mardi un appel d'urgence pour lever des dizaines de millions de dollars afin de protéger les réfugiés vulnérables face à la propagation du nouveau coronavirus.

## usage – étiquetage

Les annotations portant sur les tokens sont accessibles via les attributs des objets de type `token` : [https://spacy.io/api/token#attributes](https://spacy.io/api/token#attributes)  
  - `pos_` contient l'étiquette de partie du discours de [universal dependancies](https://universaldependencies.org/docs/u/pos/)
  - `tag_` contient l'étiquette du corpus original, parfois plus détaillée
  - `lemma_` pour le lemme
  - `morph` pour l'analyse morphologique

In [27]:
for token in doc:
    print(token.text, token.pos_, token.morph, token.lemma_)

L' DET Definite=Def|Number=Sing|PronType=Art le
Organisation NOUN Gender=Fem|Number=Sing organisation
des ADP Definite=Def|Number=Plur|PronType=Art de
Nations PROPN  Nations
unies ADJ Gender=Fem|Number=Plur uni
( PUNCT  (
ONU PROPN Gender=Fem|Number=Sing ONU
) PUNCT  )
a AUX Mood=Ind|Number=Sing|Person=3|Tense=Pres|VerbForm=Fin avoir
lancé VERB Gender=Masc|Number=Sing|Tense=Past|VerbForm=Part lancer
mardi NOUN Gender=Masc|Number=Sing mardi
un DET Definite=Ind|Gender=Masc|Number=Sing|PronType=Art un
appel NOUN Gender=Masc|Number=Sing appel
d' ADP  de
urgence NOUN Gender=Fem|Number=Sing urgence
pour ADP  pour
lever VERB VerbForm=Inf lever
des DET Definite=Ind|Number=Plur|PronType=Art un
dizaines NOUN Gender=Fem|Number=Plur dizaine
de ADP  de
millions NOUN Gender=Masc|NumType=Card|Number=Plur million
de ADP  de
dollars NOUN Gender=Masc|Number=Plur dollar
afin ADV  afin
de ADP  de
protéger VERB VerbForm=Inf protéger
les DET Definite=Def|Number=Plur|PronType=Art le
réfugiés NOUN Gender=Masc

Pour traiter plusieurs textes en série, il est recommandé d'utiliser [nlp.pipe](https://spacy.io/api/language#pipe)

In [40]:
texts = [
    "Cadine avait un très-mauvais caractère. Elle ne s’accommodait pas du rôle de servante.",
    "Aussi finit-elle par s’établir pour son compte.",
    "Comme elle était alors âgée de treize ans, et qu’elle ne pouvait rêver le grand commerce, un banc de vente de l’allée aux fleurs, elle vendit des bouquets de violettes d’un sou, piqués dans un lit de mousse, sur un éventaire d’osier pendu à son cou.",
    "Elle rôdait toute la journée dans les Halles, autour des Halles, promenant son bout de pelouse.",
    "C’était là sa joie, cette flânerie continuelle, qui lui dégourdissait les jambes, qui la tirait des longues heures passées à faire des bouquets, les genoux pliés, sur une chaise basse.",
    "Maintenant, elle tournait ses violettes en marchant, elle les tournait comme des fuseaux, avec une merveilleuse légèreté de doigts ; elle comptait six à huit fleurs, selon la saison, pliait en deux un brin de jonc, ajoutait une feuille, roulait un fil mouillé ; et, entre ses dents de jeune loup, elle cassait le fil."
]

In [41]:
docs = nlp.pipe(texts)

✍️ À vous  
1. Extrayez de la série de phrases ci-dessus la liste des noms communs
2. Comptez le nombre de tokens au masculin et au féminin

In [34]:
# Extrayez de la série de phrases ci-dessus la liste des noms communs

ncs = []
docs = nlp.pipe(texts)
for doc in docs:
    for token in doc:
        if token.pos_ == "NOUN":
            ncs.append(token)

ncs_set = set(ncs)
#for item in ncs:
#    print(item.text)
print(", ".join([item.text for item in ncs_set]))

chaise, commerce, fil, banc, bouquets, violettes, fil, fuseaux, mousse, rôle, compte, bout, heures, osier, saison, brin, ans, feuille, vente, allée, violettes, -, sou, légèreté, servante, Halles, pelouse, jambes, bouquets, fleurs, genoux, dents, fleurs, loup, Cadine, caractère, joie, journée, flânerie, lit, éventaire, Halles, doigts, cou


In [39]:
# Comptez le nombre de tokens au masculin et au féminin

nb_masc = 0
nb_fem = 0

docs = nlp.pipe(texts)
for doc in docs:
    for token in doc:
        if token.morph.get("Gender") == ["Masc"]:
            nb_masc += 1
        elif token.morph.get("Gender") == ["Fem"]:
            nb_fem += 1
            
print(f"tokens au masculin : {nb_masc}, tokens au féminin : {nb_fem}")

tokens au masculin : 39, tokens au féminin : 47


## usage – NER

Si NER (*Named Entity Recognition*) fait partie de votre modèle, vos données seront annotées également en entités nommées.  
Vous pouvez y accéder avec l'attribut `ent_type_` des tokens

In [44]:
doc = nlp("L'Organisation des Nations unies (ONU) a lancé mardi un appel d’urgence pour lever des dizaines de millions de dollars afin de protéger les réfugiés vulnérables face à la propagation du nouveau coronavirus.")
for token in doc:
    print(token, token.ent_type_)

L' 
Organisation ORG
des ORG
Nations ORG
unies ORG
( 
ONU ORG
) 
a 
lancé 
mardi 
un 
appel 
d’ 
urgence 
pour 
lever 
des 
dizaines 
de 
millions 
de 
dollars 
afin 
de 
protéger 
les 
réfugiés 
vulnérables 
face 
à 
la 
propagation 
du 
nouveau 
coronavirus 
. 


Ou accéder directement aux entités de l'objet `Doc`

In [55]:
nlp = spacy.load("fr_core_news_md")
doc = nlp("L’Organisation des Nations unies (ONU) a lancé mardi un appel d’urgence pour lever des dizaines de millions de dollars afin de protéger les réfugiés vulnérables face à la propagation du nouveau coronavirus.")
for ent in doc.ents:
    print(ent.text, ent.label_)

Organisation des Nations unies ORG
ONU ORG


Spacy intègre un outil de visualisation pour l'annotation en entités nommées :

In [46]:
from spacy import displacy
displacy.render(doc, style="ent", jupyter=True)

In [47]:
doc = nlp('Le président Xi Jinping a affirmé que la propagation du coronavirus était « pratiquement jugulée ». Il s’est d’ailleurs rendu pour la première fois à Wuhan, la capitale de la province du Hubei, le berceau du Covid-19.')
displacy.render(doc, style="ent", jupyter=True)

In [61]:
doc = nlp("Derrière lui, sur le carreau de la rue Rambuteau, on vendait des fruits.")
displacy.render(doc, style="ent", jupyter=True)

✍️ À vous  

Dans `files/Le_Ventre_de_Paris-short.txt` (ou un texte de votre choix), comptez la fréquence de chaque entité de type PER.

In [63]:
from collections import Counter

personnes = Counter()
with open('files/Le_Ventre_de_Paris-short.txt') as input_data:
    docs = nlp.pipe(input_data)
    for doc in docs:
        for ent in doc.ents:
            if ent.label_ == "PER":
                personnes[ent.text] += 1

print(personnes.most_common())

[('Florent', 218), ('Lisa', 76), ('Claude', 33), ('Pauline', 22), ('Saget', 18), ('Lecœur', 18), ('Auguste', 18), ('Augustine', 16), ('François', 15), ('Charvet', 15), ('Lebigre', 13), ('Léon', 13), ('Robine', 11), ('Méhudin', 10), ('madame Quenu', 9), ('Gradelle', 8), ('Clémence', 8), ('Balthazar', 6), ('Alexandre', 6), ('Madame François', 5), ('Chantemesse', 5), ('Jules', 5), ('Marjolin', 5), ('Claude Lantier', 5), ('Quenu', 5), ('Collard', 5), ('Louise', 5), ('Claire', 5), ('Murillo', 4), ('Macquart', 4), ('Mademoiselle Saget', 4), ('Verlaque', 4), ('Hein', 3), ('Voyons', 3), ('Cadine', 3), ('Madame Lecœur', 3), ('Gavard', 3), ('Rose', 3), ('Augustine Landois', 2), ('Charles X', 2), ('Louis-Philippe', 2), ('Madame Taboureau', 2), ('Mouton', 2), ('Doucement', 2), ('Échappé de Cayenne', 1), ('Marcel', 1), ('Dis Marcel', 1), ('Lacaille', 1), ('Lundi', 1), ('Adieu', 1), ('Cendrillon', 1), ('Rubens', 1), ('Guillout', 1), ('Aveuglé', 1), ('de Florent', 1), ('Cuvier', 1), ('Gavard voulut',

## usage – analyse syntaxique

L'analyse syntaxique ou *parsing* de Spacy est une analyse en dépendance. La plupart sinon la totalité des modèles utilisés viennent de https://universaldependencies.org

Dans l'analyse en dépendance produite par Spacy, chaque mot d'une phrase a un gouverneur unique (*head*), la relation de dépendance entre le mot et son gouverneur est typée (*nsubj*, *obj*, …).  
Pour la tête de la phrase on utilise la relation *ROOT*.

La structure produite par l'analyse syntaxique est un arbre, un graphe acyclique et connexe. Les tokens sont les nœuds, les arcs sont les dépendances, le type de la relation est l'étiquette de l'arc.

`displacy` fournit un outil de visualisation bien pratique :

In [23]:
doc = nlp("Derrière lui, sur le carreau de la rue Rambuteau, on vendait des fruits.")
displacy.render(doc, style="dep", jupyter=True, options={'distance':90})

Il existe également un outil issu d'un développement indépendant : [explacy](https://spacy.io/universe/project/explacy)

In [24]:
import explacy
explacy.print_parse_info(nlp, 'Derrière lui, sur le carreau de la rue Rambuteau, on vendait des fruits.')

Dep tree     Token     Dep type Lemma     Part of Sp
──────────── ───────── ──────── ───────── ──────────
         ┌─► Derrière  case     derrière  ADP       
┌───────►└── lui       obl:mod  lui       PRON      
│┌─────────► ,         punct    ,         PUNCT     
││      ┌──► sur       case     sur       ADP       
││      │┌─► le        det      le        DET       
││┌─►┌──┴┴── carreau   obl:mod  carreau   NOUN      
│││  │  ┌──► de        case     de        ADP       
│││  │  │┌─► la        det      le        DET       
│││  └─►└┼── rue       nmod     rue       NOUN      
│││      └─► Rambuteau nmod     Rambuteau PROPN     
│││     ┌──► ,         punct    ,         PUNCT     
│││     │┌─► on        nsubj    on        PRON      
└┴┴──┬┬─┴┴── vendait   ROOT     vendre    VERB      
     ││  ┌─► des       det      un        DET       
     │└─►└── fruits    obj      fruit     NOUN      
     └─────► .         punct    .         PUNCT     


On peut aussi récupérer parcourir les tokens et afficher 

In [16]:
for token in doc:
    print(token, token.dep_.upper(), token.head)

Maintenant ADVMOD tournait
, PUNCT tournait
elle NSUBJ tournait
tournait ROOT tournait
ses DET violettes
violettes OBJ tournait
en CASE marchant
marchant ADVCL tournait
, PUNCT tournait
elle NSUBJ tournait
les OBJ tournait
tournait CONJ tournait
comme CASE fuseaux
des DET fuseaux
fuseaux OBL:MOD tournait
, PUNCT tournait
avec CASE légèreté
une DET légèreté
merveilleuse AMOD légèreté
légèreté OBL:MOD tournait
de CASE doigts
doigts NMOD légèreté
; PUNCT tournait
elle NSUBJ comptait
comptait ROOT comptait
six NUMMOD fleurs
à CASE huit
huit NMOD six
fleurs OBJ comptait
, PUNCT comptait
selon CASE saison
la DET saison
saison OBL:MOD comptait
, PUNCT pliait
pliait CONJ comptait
en CASE deux
deux OBL:MOD pliait
un DET brin
brin OBJ pliait
de CASE jonc
jonc NMOD brin
, PUNCT ajoutait
ajoutait CONJ comptait
une DET feuille
feuille OBJ ajoutait
, PUNCT roulait
roulait CONJ comptait
un DET fil
fil OBJ roulait
mouillé AMOD fil
; PUNCT comptait
et CC cassait
, PUNCT cassait
entre CASE dents
ses DET

Les attributs de token suivant peuvent être utilisés pour parcourir l'arbre de dépendance : 
- `children` les tokens dépendants du token
- `subtree` tous les descendants du token
- `ancestors` tous les parents du token
- `rights` les enfants à droite du token
- `lefts` les enfants à gauche du token

On peut extraire de la phrase précédente le triplet sujet-verbe-objet comme ceci :

In [25]:
root = [token for token in doc if token.head == token][0]
subjects = [tok for tok in root.lefts if tok.dep_ == "nsubj"]
subject = subjects[0]
objs = [tok for tok in root.rights if tok.dep_ == "obj"]
obj = objs[0]
subject, root, obj

(on, vendait, fruits)

In [26]:
for obj in objs:
    for descendant in obj.subtree:
        print(descendant.text)

des
fruits


✍️ À vous

1. Trouver et afficher l'objet de la phrase : « Depuis que Google a annoncé son intention de stopper d'ici deux ans les cookies tiers sur Chrome , son moteur de recherche qui est utilisé par plus de 60 % de la population mondiale connectée, les Criteo, LiveRamp et autres Index Exchange se préparent à ce qui peut être considéré comme un séisme, à leur échelle. »

2. Dans la liste `texts` vue plus haut, retrouvez les verbes dont Cadine ou le pronom 'elle' est sujet.

In [130]:
import explacy
explacy.print_parse_info(nlp, "Depuis que Google a annoncé son intention de stopper d'ici deux ans les cookies tiers sur Chrome , son moteur de recherche qui est utilisé par plus de 60 % de la population mondiale connectée, les Criteo, LiveRamp et autres Index Exchange se préparent à ce qui peut être considéré comme un séisme, à leur échelle.")

Dep tree                         Token      Dep type   Lemma      Part of Sp
──────────────────────────────── ────────── ────────── ────────── ──────────
                          ┌────► Depuis     mark       depuis     ADP       
                          │┌───► que        mark       que        SCONJ     
                          ││┌──► Google     nsubj      Google     PROPN     
                          │││┌─► a          aux:tense  avoir      AUX       
┌─►┌──────────────────────┴┴┴┴── annoncé    advcl      annoncer   VERB      
│  │                         ┌─► son        det        son        DET       
│  └─►┌──────────────────────┴── intention  obj        intention  NOUN      
│     │                      ┌─► de         mark       de         ADP       
│     └─►┌───────────────────┴── stopper    acl        stopper    VERB      
│        │                ┌─►┌── d'         advmod     de         ADP       
│        │                │  └─► ici        fixed      ici        ADV       

In [30]:
doc = nlp("Depuis que Google a annoncé son intention de stopper d'ici deux ans les cookies tiers sur Chrome , son moteur de recherche qui est utilisé par plus de 60 % de la population mondiale connectée, les Criteo, LiveRamp et autres Index Exchange se préparent à ce qui peut être considéré comme un séisme, à leur échelle.")
root = [token for token in doc if token.head == token][0]
objs = [tok for tok in root.rights if tok.dep_ in ("obl:arg", "obj", "iobj")]
for obj in objs:
    for descendant in obj.subtree:
        print(descendant.text_with_ws, end="")

à ce qui peut être considéré comme un séisme, à leur échelle

## Matching

## 1. Matching par règle

Spacy a une classe `Matcher` qui permet de repérer des tokens ou des suites de tokens à l'aide de patrons (*pattern*). Ces patrons peuvent porter sur la forme des tokens ou leurs attributs (pos, ent).  
On peut aussi utiliser des catégories comme `IS_ALPHA` ou `IS_NUM`, voir la [doc](https://spacy.io/usage/rule-based-matching#adding-patterns-attributes)

In [77]:
from spacy.matcher import Matcher

matcher = Matcher(nlp.vocab)
pattern = [{"LOWER": "en"}, {"LOWER": "taille"}, {"IS_ALPHA": True, "IS_UPPER": True}]
# en taille + lettres en maj
matcher.add("tailles", [pattern])

doc = nlp("Ce modèle est aussi disponible en taille XL ; je vous le conseille.")
matches = matcher(doc)
for _, start, end in matches:
    #string_id = nlp.vocab.strings[match_id]  # Get string representation
    span = doc[start:end]  # The matched span
    print(start, end, span.text)

5 8 en taille XL


Ça fonctionne pour les séquences comme « en taille M » ou « en taille XL » mais pas pour « vous l'avez en XL ? »

In [72]:
doc = nlp("vous l'avez en XL ?")
matches = matcher(doc)
for match_id, start, end in matches:
    string_id = nlp.vocab.strings[match_id]  # Get string representation
    span = doc[start:end]  # The matched span
    print(match_id, string_id, start, end, span.text)

On peut essayer d'améliorer les règles :

In [73]:
matcher = Matcher(nlp.vocab)
pattern_1 = [{"LOWER": "en"}, {"LOWER": "taille"}, {"IS_ALPHA": True, "IS_UPPER": True}]
pattern_2 = [{"LOWER": "en"}, {"IS_ALPHA": True, "IS_UPPER": True}]
matcher.add("tailles", [pattern_1, pattern_2])
# règle avec deux patterns

doc = nlp("vous l'avez en XL ?")
matches = matcher(doc)
for _, start, end in matches:
    #string_id = nlp.vocab.strings[match_id]  # Get string representation
    span = doc[start:end]  # The matched span
    print(span.text)

en XL


Ou encore :

In [74]:
matcher = Matcher(nlp.vocab)
sizes = ['XS', 'S', 'M', 'L', 'XL']
pattern_1 = [{"LOWER": "en"}, {"LOWER": "taille"}, {"TEXT": {"IN": sizes}}]
pattern_2 = [{"LOWER": "en"}, {"TEXT": {"IN": sizes}}]
matcher.add("tailles", [pattern_1, pattern_2])
# règle avec deux patterns

doc = nlp("vous l'avez en XL ?")
matches = matcher(doc)
for match_id, start, end in matches:
    string_id = nlp.vocab.strings[match_id]  # Get string representation
    span = doc[start:end]  # The matched span
    print(match_id, string_id, start, end, span.text)

9364735015326875510 tailles 3 5 en XL


✍️ À vous

Dans `files/Le_Ventre_de_Paris-short.txt`, trouver les séquences pronom - le lemme 'vendre'

In [78]:
matcher = Matcher(nlp.vocab)
pattern = [{"POS": "PRON"}, {"LEMMA": "vendre"}]
matcher.add("pro_vendre", [pattern])

doc = ""
with open('files/Le_Ventre_de_Paris-short.txt') as input_data:
    doc = nlp(input_data.read())
matches = matcher(doc)

In [79]:
for _, start, end in matches:
    #string_id = nlp.vocab.strings[match_id]  # Get string representation
    span = doc[start:end]  # The matched span
    print(span.text)

elle vend
on vendait
qui vendait
qui vendaient
elle vendait
Il vendrait


## 4. Dependency Matcher : extraction de patrons

Depuis la v3, Spacy a ajouté un *Dependancy Matcher* qui permet de faire de l'extraction de patrons syntaxiques. Il est maintenant possible de faire porter des requêtes sur l'arbre syntaxique et non plus seulement sur la séquence des tokens.  
Ce dispositif utilise [Semgrex](https://nlp.stanford.edu/nlp/javadoc/javanlp/edu/stanford/nlp/semgraph/semgrex/SemgrexPattern.html), la syntaxe utilisée dans Tgrep et Tregex, les outils de requête sur Treebank de Stanford.

Voir la [documentation](https://spacy.io/usage/rule-based-matching#dependencymatcher)

In [80]:
ventre_short = ""
with open('files/Le_Ventre_de_Paris-short.txt') as input_f:
    ventre_short = input_f.read()
doc = nlp(ventre_short)

In [81]:
from spacy.matcher import DependencyMatcher

matcher = DependencyMatcher(nlp.vocab)
pattern = [
  {
    "RIGHT_ID": "vendre",    
    "RIGHT_ATTRS": {"LEMMA": "vendre"}
  }
]
matcher.add("VENDRE", [pattern])
matches = matcher(doc)
for m_id, t_ids in matches:
    for t_id in t_ids:
        print(doc[t_id])

vend
vendant
vendait
vendait
vendaient
vendaient
vend
vendu
vendu
vendre
vendait
vendu
vendais
vendu
vendrait


In [83]:
from spacy.matcher import DependencyMatcher

matcher = DependencyMatcher(nlp.vocab)
pattern = [
    {
        "RIGHT_ID": "vendre",    
        "RIGHT_ATTRS": {"LEMMA": {"IN": ["vendre", "acheter"]}}
    },
    {
        "LEFT_ID": "vendre",
        "REL_OP": ">",
        "RIGHT_ID": "sujet",
        "RIGHT_ATTRS": {"DEP": "nsubj"},  
    },
    {
        "LEFT_ID": "vendre",
        "REL_OP": ">",
        "RIGHT_ID": "objet",
        "RIGHT_ATTRS": {"DEP": {"IN": ["obj", "iobj", "obl"]}},  
    }
]
matcher.add("VENDRE", [pattern])
matches = matcher(doc)
for m_id, t_ids in matches:
    print("verbe, sujet, objet : ", " -> ".join([doc[t_id].text for t_id in t_ids]))
    print("objet complet : ", " ".join([t.text for t in doc[t_ids[2]].subtree]))
    print("Phrase compléte : ", doc[t_ids[0]].sent)
    print()

verbe, sujet, objet :  acheta -> il -> derniers
objet complet :  ses deux derniers sous de pain
Phrase compléte :  Mais, à Vernon, il acheta ses deux derniers sous de pain.

verbe, sujet, objet :  achetait -> elle -> qu’
objet complet :  qu’
Phrase compléte :  J’étais gamine, qu’elle achetait déjà ses navets à mon père.

verbe, sujet, objet :  achetait -> elle -> navets
objet complet :  ses navets à mon père
Phrase compléte :  J’étais gamine, qu’elle achetait déjà ses navets à mon père.

verbe, sujet, objet :  vendait -> on -> fruits
objet complet :  des fruits
Phrase compléte :  

Derrière lui, sur le carreau de la rue Rambuteau, on vendait des fruits.

verbe, sujet, objet :  vendaient -> qui -> bottes
objet complet :  des bottes de fougère et des paquets de feuilles de vigne , bien réguliers , attachés par quarterons
Phrase compléte :  Ils s’arrêtèrent curieusement devant des femmes qui vendaient des bottes de fougère et des paquets de feuilles de vigne, bien réguliers, attachés par 

✍️ À vous

Ajouter une règle au motif pour trouver aussi l'objet

## Adapter les traitements de Spacy

## 1. re-tokenisation

- voir [https://spacy.io/usage/linguistic-features#retokenization](https://spacy.io/usage/linguistic-features#retokenization)

Dans l'exemple qui suit « quer-cra » sera tokenizé à tort.

In [3]:
doc = nlp("Pour les bons bails ça va grave quer-cra")
print([(tok.text, tok.pos_, tok.lemma_)for tok in doc])

[('Pour', 'ADP', 'pour'), ('les', 'DET', 'le'), ('bons', 'ADJ', 'bon'), ('bails', 'NOUN', 'bail'), ('ça', 'PRON', 'cela'), ('va', 'VERB', 'aller'), ('grave', 'ADJ', 'grave'), ('quer', 'VERB', 'quer'), ('-', 'PUNCT', '-'), ('cra', 'PROPN', 'cra')]


In [4]:
with doc.retokenize() as retokenizer:
    retokenizer.merge(doc[7:], attrs={"LEMMA": "quer-cra", "POS": "NOUN"})
print([(tok.text, tok.pos_) for tok in doc])

[('Pour', 'ADP'), ('les', 'DET'), ('bons', 'ADJ'), ('bails', 'NOUN'), ('ça', 'PRON'), ('va', 'VERB'), ('grave', 'ADJ'), ('quer-cra', 'NOUN')]


Attention ici c’est l’objet doc qui est modifié, le résultat mais pas le traitement. Nous allons voir comment faire pour modifier le traitement.

## 2. Modification de la tokenisation

In [5]:
from spacy.symbols import ORTH, LEMMA, POS, TAG

special_case = [{ORTH: "quer-cra"}]
nlp.tokenizer.add_special_case("quer-cra", special_case)
doc = nlp("Pour les bons bails ça va grave quer-cra")
print([(tok.text, tok.pos_, tok.lemma_) for tok in doc])

[('Pour', 'ADP', 'pour'), ('les', 'DET', 'le'), ('bons', 'ADJ', 'bon'), ('bails', 'NOUN', 'bail'), ('ça', 'PRON', 'cela'), ('va', 'VERB', 'aller'), ('grave', 'ADJ', 'grave'), ('quer-cra', 'PROPN', 'quer-cra')]


On a bien modifié la tokenisation dans le modèle `nlp`. Cela n'affecte pas par contre l'étiquetage en POS.

## 3. Entités nommées : traitement par règles
 - Voir [https://spacy.io/usage/rule-based-matching#entityruler](https://spacy.io/usage/rule-based-matching#entityruler)
 
Spacy offre aussi un mécanisme de traitement par règle pour les entités nommées

In [6]:
from spacy.pipeline import EntityRuler

#nlp = spacy.load('fr_core_news_md')
doc = nlp("Depuis que machin a annoncé son intention de stopper d'ici deux ans les cookies tiers sur Chrome , son moteur de recherche qui est utilisé par plus de 60 % de la population mondiale connectée, les Criteo, LiveRamp et autres Index Exchange se préparent à ce qui peut être considéré comme un séisme, à leur échelle.")
print("Avant : ", [(ent.text, ent.label_) for ent in doc.ents])


ruler = nlp.add_pipe("entity_ruler", config={'overwrite_ents':True})
patterns = [{"label": "ORG", "pattern": "Chrome"},
            {"label":"ORG", "pattern":"machin"},
    {"label":"ORG", "pattern":"Criteo"},
    {"label":"ORG","pattern":"LiveRamp"}]
ruler.add_patterns(patterns)

doc = nlp("Depuis que machin a annoncé son intention de stopper d'ici deux ans les cookies tiers sur Chrome , son moteur de recherche qui est utilisé par plus de 60 % de la population mondiale connectée, les Criteo, LiveRamp et autres Index Exchange se préparent à ce qui peut être considéré comme un séisme, à leur échelle.")
print("Après : ", [(ent.text, ent.label_) for ent in doc.ents])

Avant :  [('Chrome', 'MISC'), ('Criteo', 'MISC'), ('LiveRamp', 'ORG'), ('Index Exchange', 'MISC')]
Après :  [('machin', 'ORG'), ('Chrome', 'ORG'), ('Criteo', 'ORG'), ('LiveRamp', 'ORG'), ('Index Exchange', 'MISC')]
